In [1]:
import numpy as np
import sounddevice as sd
import datetime
from scipy.signal import butter, lfilter
from scipy.io import wavfile

def generate_white_noise(duration, sample_rate=44100):
    """"""
    white_noise = np.random.randn(int(sample_rate * duration))
    return white_noise

def band_limited_white_noise(low, high, duration, sample_rate=44100):
    """Apply a butterworth filter"""
    white_noise = generate_white_noise(duration, sample_rate)
    nyq = 0.5 * sample_rate
    low = low / nyq
    high = high / nyq
    b, a = butter(5, [low, high], btype='band')
    band_limited = lfilter(b, a, white_noise)
    return band_limited

def pseudo_white_noise(low, high, repetitions, duration, sample_rate=44100):
    """Generate pseudoNoise"""
    # use sinusoidal signal with random phase as pseudo noise
    # t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    # pseudo_noise = np.sin(2 * np.pi * 20 * t + np.random.uniform(0, 2 * np.pi))
    band_limited_noise = band_limited_white_noise(low, high, duration/repetitions)
    pseudo_noise = np.tile(band_limited_noise, repetitions)
    # store the sound signal
    # Format the date and time
    filename = f"pseudoNoise_{low}_{high}.wav"
    wavfile.write(filename, sample_rate, pseudo_noise)
    return pseudo_noise

def play_sound(sound, sample_rate=44100):
    """Play the sound"""
    sd.play(sound, samplerate=sample_rate)
    sd.wait()

ModuleNotFoundError: No module named 'sounddevice'

In [3]:

# generate and play white noise
# for i in np.arange(10):
duration = 100  # 10秒
band_limited_noise = band_limited_white_noise(50, 8000, duration)
play_sound(band_limited_noise)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# parameters：
sample_rate=44100
time_signal = np.arange(0, duration, 1/sample_rate)
data_signal = band_limited_noise 

# FFT
fft_result = np.fft.fft(data_signal)
frequencies = np.fft.fftfreq(len(fft_result), d=(time_signal[1] - time_signal[0]))  # 计算频率

# choose the frequency range of excitation signal
min_freq = 0
max_freq = 16000
mask = (frequencies >= min_freq) & (frequencies <= max_freq)

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time_signal, data_signal)
plt.title('Original Signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(frequencies[mask], np.abs(fft_result[mask]))
plt.title('Magnitude Spectrum')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude')
plt.tight_layout()
plt.show()


NameError: name 'duration' is not defined

In [25]:
# Generate pseudo white noise
repetitions = 5
duration = 10
pseudo_noise = pseudo_white_noise(2000, 4000, repetitions, duration)

In [27]:
# Play pseudo White Noise
sample_rate, pseudo_noise = wavfile.read("pseudoNoise_2000_4000.wav")

repetitions = 1
pseudo_noise = np.tile(pseudo_noise, repetitions)
play_sound(pseudo_noise)
